# SCRAPANDO alerta licitação GO

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import seaborn as sns
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
##from selenium import webdriver


In [3]:
def trata_html(input):
    html = input.decode('utf-8')
    return " ".join(html.split()).replace('> <', '><')

### Class_="panel" é a tag que mantem todos os dados de cada licitacão

In [4]:
#camila = joule.findAll(class_="panel")
#camila[x].get_text(separator=' ', strip=True)

## Selecionando apenas as datas 
#### as datas são o penultimo item das class_="panel"

 pesquisando por meio de regex, encontra a formula dentre as as strings

In [5]:
def cronis(camilap): # pesquisando por meio de regex, encontra a formula dentre as as strings
    terminus = []
    for i in camilap:
        ptext = (i.findAll("p")[2:-1])
        for l in ptext: 
            result = re.search(r"\d\d/\d\d/\d\d\d\d", i.get_text())
        terminus.append(result.group())
    return(terminus)

# Objetos

In [6]:
def obje(camila):
    objetivo = []
    for i in camila:
        objetivo.append(i.findAll('p')[-1].get_text())
    return(objetivo)
        

## Selecionando apenas nomes.
#### Os nomes estao entre h3 na  class_="panel"

In [7]:
def nama(camila):
    nome = []
    for i in camila:
        x=(i.find("h3").get_text().split("(")[-2])
        nome.append(x)
    return nome
    

## Cidades nomes:
#### Estao dispersas e sem classe, mas vem escritas municipio ou cidade antes do nome, transforma cada linha e lista, achar
#### a palavra chave municipio ou cidade e pegar a ultima palavra que sera o nome

In [8]:
def cidadela(oio):
    cidade = "nada"
    ei = []
    for i in oio:
        ei.append(i.get_text().split(":"))
    ei = np.array(ei[:-2])
    for i in ei:
        for j in i:
            if j == "Cidade":
                cidade = (i[-1])
            elif j == "Municipio":
                cidade = ((i[-1]))
            cidade = (cidade.split("/"))[0]
            cidade = (cidade.split("-"))[0]
            cidade = (cidade.split("("))[0]
            cidade = cidade.strip()
            cidade = cidade + " - " + go
    return(cidade)
#city = cidadela(cacu)
#city

In [9]:
def omnicity (camila):  
    lacunas=[]
    for i in camila:
        z = i.findAll("p")
        z=cidadela(z)
        lacunas.append(z)
    return lacunas

# Nome, Cidade, Tempo, Descrição e Pandas v0.22

In [10]:
def goianando(est):
    tempo = []
    descri = []
    nome = []
    cidade = []
    for k in range(10):
        uai = ("https://alertalicitacao.com.br//ajax/e.php?e="+str(est)+"&paginaAjax="+ str(k))
        response = urlopen(uai)
        html = response.read()
        goin = trata_html(html)
        uai = BeautifulSoup(goin, 'html.parser')
        #Area onde se encontra os paneis com as licitacoes
        camila = uai.findAll(class_="panel")
        tempo += cronis(camila)
        descri += obje(camila)
        cidade += omnicity(camila)  
        nome += nama(camila)
    dicionario = {"nome":nome,"cidade":cidade,"tempo":tempo,"Description":descri}
    return (dicionario)

In [11]:
go="GO" # Sigla maiuscula do estado de interesse
lorenzo = goianando(go)

C:\Users\Micro\AppData\Local\Temp/ipykernel_13836/2346625263.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ei = np.array(ei[:-2])


In [12]:
tabuleta = pd.DataFrame(lorenzo)

In [13]:
tabuleta["tempo"] = pd.to_datetime(tabuleta["tempo"],format = "%d/%m/%Y")
tabuleta.sort_values(by = ['tempo'],inplace = True)
tabuleta.reset_index(drop=True, inplace = True)
tabuleta["tempo"] = [i.strftime("%d-%m") for i in tabuleta.tempo]

# Função de pesquisa no DataFrame:

Criando conluna com keywords, formando lista para ser chamado no proximo def

In [14]:
def keywords(frame,obj1,obj2):
    tipo =[]
    separador = " em "
    for i in frame.Description:
        keyword = []
        telo = "obra"       #telo = re.search(obj1,i,re.IGNORECASE).group()
        nelo =re.search(obj2,i,re.IGNORECASE).group()
        keyword = [telo,nelo]
        tipo.append(separador.join(keyword).lower())
    return(tipo)

Procurando no data frame pelas keywords = cidade, servico 1(tipo ex: reforma) - servico 2( de que, ex: escola)  

In [15]:
def pesquisa(frame,city,string,string2):
    z=0
    if city == 0:
        frame = frame
    else:
        frame = frame[frame.cidade.str.contains(city, case=False, regex=True)]
    if string == 0:
        frame = frame
        z+=1
    else:
        frame = frame[frame.Description.str.contains(string, case=False, regex=True)]
    if string2 == 0:
        frame = frame
        z+=1
    else:
        frame = frame[frame.Description.str.contains(string2, case=False, regex=True)]
    if z==0:
        keyword=keywords(frame,string,string2)
        frame.insert(3,"palavra_chave",keyword) 
    return frame
    

Demonstrar em texto corrido as licitaçoes

In [16]:
obj1 = 'construção|reforma|reconstrução|obras|obra'
obj2 = "escola|creche|estádio|Unidade Basica de Saúde|Calçada|Calçadas|cmei|ubs"
cidades = 'goiânia|luziânia|Formosa|Valparaíso|planaltina'
resumo = pesquisa(tabuleta,0,obj1,obj2).reset_index(drop=True)

# Analisando

In [17]:
resumo

,nome,cidade,tempo,palavra_chave,Description
0,PREGÃO ELETRÔNICO Nº 6/2022,Chapadão do Céu - GO,01-02,obra em ubs,AVISO DE licitaçãoPREGÃO ELETRÔNICO Nº 6/2022...
1,TOMADA DE PREÇOS PARA OBRAS E SERVIÇOS DE ENGE...,Abadia de Goiás - GO,01-02,obra em escola,SOLICITAÇÃO DE CONTRATAÇÃO DE EMPRESA ESPECIA...
2,Pregão Eletrônico Nº 3/2022,Turvânia - GO,02-02,obra em escola,"Pregão Eletrônico - Aquisição de cimento, mat..."
3,TOMADA DE PREÇOS Nº 1/2022,Goianira - GO,02-02,obra em ubs,TOMADA DE PREÇOS Nº 1/2022 O Município de Goi...
4,TOMADA DE PREÇOS Nº 1/2022,Faina - GO,02-02,obra em estádio,TOMADA DE PREÇOS Nº 1/2022 O Município de Fai...
5,TOMADA DE PREÇOS N° 1/2022,Abadia de Goiás - GO,02-02,obra em escola,TOMADA DE PREÇOS N° 1/2022 O Município de Aba...
6,Tomada de preços 1/2022,Trindade - GO,02-02,obra em escola,CONTRATAÇÃO DE EMPRESA DE ENGENHARIA PARA CON...
7,TOMADA DE PREÇOS Nº 1/2022,Doverlândia - GO,03-02,obra em escola,Contratação de empresa para reforma e ampliaç...
8,TOMADA DE PREÇOS Nº 1/2022,Cachoeira Dourada - GO,04-02,obra em escola,AVISO DE LICITAÇÃOTOMADA DE PREÇOS Nº 1/2022 ...
9,Tomada de preços 1/2022,Campestre de Goiás - GO,04-02,obra em creche,CONTRATAÇÃO DE EMPRESA PARA REALIZAR A REFORM...
